In [ ]:
# Importo librerías a usar
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

# Leo el Bearer token desde la variable de entorno
BEARER_TOKEN = os.getenv('BEARER_TOKEN')
if BEARER_TOKEN is None:
    raise ValueError("No se encontró la variable BEARER_TOKEN en el archivo .env")

# Defino el encabezado de petición HTTP
headers = {
    'Authorization': f'Bearer {BEARER_TOKEN}'
}

# Defino items a buscar
search_terms = ['chromecast', 'google home', 'apple tv', 'amazon fire tv', 'roku streaming']

# Declaro URLs de las APIs a consultar
base_search_url = 'https://api.mercadolibre.com/sites/MLA/search?q={}&limit=50' # URL buscador
base_item_url = 'https://api.mercadolibre.com/items/{}' # URL detalle de ítem

# Defino atributos únicos de cada producto importantes a retener
important_attributes = ['OS_NAME', 'MAX_VIDEO_RESOLUTION', 'STORAGE_CAPACITY', 'RAM_MEMORY', 
                        'CPU', 'INTEGRATED_VOICE_ASSISTANTS', 'REMOTE_CONTROL_TYPE', 'PORTS_AND_CONNECTORS_TYPES', 
                        'WITH_BLUETOOTH', 'WITH_WI_FI', 'WI_FI_STANDARDS']

results_list = []

# Itero sobre cada item a buscar
for term in search_terms:
    # Hago una petición GET en el buscador principal de Mercado Libre
    response = requests.get(base_search_url.format(term) + "#json", headers=headers).json()
    
    # Extraigo los item_id de cada producto obtenido
    item_ids = [item['id'] for item in response.get('results', [])]

    # Itero sobre cada item para obtener sus datos
    for item_id in item_ids:
        # Hago la petición GET a la API para obtener detalles del item
        item = requests.get(base_item_url.format(item_id), headers=headers).json()

        # Extraigo información básica del producto
        seller = item.get('seller', {})
        address = item.get('seller_address', {})
        shipping = item.get('shipping', {})

        # Extraigo atributos clave
        item_attributes = {attr['id']: attr['value_name'] for attr in item.get('attributes', []) if attr['id'] in important_attributes}

        # Extraigo información de garantía
        warranty_time = next((term["value_name"] for term in item.get("sale_terms", []) if term["id"] == "WARRANTY_TIME"), "No especificado")
        warranty_type = next((term["value_name"] for term in item.get("sale_terms", []) if term["id"] == "WARRANTY_TYPE"), "No especificado")

        # Armar diccionario con los datos relevantes de cada item
        item_data = {
            # Datos Generales
            'id': item.get('id'),
            'title': item.get('title', "No especificado"),
            'category': item.get('category_id', "No especificado"),
            'condition': item.get('condition', "No especificado"),
            # Precio y Descuentos
            'price': item.get('price', None),
            'base_price': item.get('base_price', None),
            'discount_percentage': (
                round(((item.get('base_price') or 0) - (item.get('price') or 0)) / (item.get('base_price') or 1) * 100, 2)
                if item.get('base_price') else None
            ),
            'currency': item.get('currency_id', "No especificado"),
            # Disponibilidad y Stock
            'stock': item.get('initial_quantity', None),
            'max_purchase_quantity': next((term["value_name"] for term in item.get("sale_terms", []) if term["id"] == "PURCHASE_MAX_QUANTITY"), None),
            # Envío y Ubicación
            'free_shipping': shipping.get("free_shipping", None),
            'logistic_type': shipping.get("logistic_type", "No especificado"),
            'pickup_available': shipping.get("store_pick_up", None),
            'city': address.get("city", {}).get("name", "No especificado"),
            'state': address.get("state", {}).get("name", "No especificado"),
            'country': address.get("country", {}).get("name", "No especificado"),
            # Características técnicas
            'operating_system': item_attributes.get('OS_NAME', "No especificado"),
            'video_resolution': item_attributes.get('MAX_VIDEO_RESOLUTION', "No especificado"),
            'storage': item_attributes.get('STORAGE_CAPACITY', "No especificado"),
            'ram': item_attributes.get('RAM_MEMORY', "No especificado"),
            'cpu': item_attributes.get('CPU', "No especificado"),
            'voice_assistant': item_attributes.get('INTEGRATED_VOICE_ASSISTANTS', "No especificado"),
            'remote_type': item_attributes.get('REMOTE_CONTROL_TYPE', "No especificado"),
            'ports': item_attributes.get('PORTS_AND_CONNECTORS_TYPES', "No especificado"),
            'bluetooth': item_attributes.get('WITH_BLUETOOTH', "No especificado"),
            'wifi': item_attributes.get('WITH_WI_FI', "No especificado"),
            'wifi_standards': item_attributes.get('WI_FI_STANDARDS', "No especificado"),
            # Garantía
            'warranty_time': warranty_time,
            'warranty_type': warranty_type
        }

        # Agrego el diccionario de cada ítem a una lista
        results_list.append(item_data)

# Convierto la lista en un dataframe de Pandas
df = pd.DataFrame(results_list)

# Exporto el dataframe a un CSV
df.to_csv('mercadolibre_items.csv', index=False)
